# Backtest uitvoeren

Op 2020 data


In [27]:
# importeren packages
import pandas as pd
import numpy as np
#conda install -c anaconda py-xgboost #installeer XGBoost
import xgboost
#from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler 
import pickle
from matplotlib import pyplot
#import seaborn as sns #visualisation
#import matplotlib.pyplot as plt #visualisation

# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"


#%matplotlib inline

In [85]:
# display meer regels
pd.set_option('display.max_rows', 500)

### Inladen data

In [54]:
# # inladen bestand met scores als sc
# sc = pd.read_excel(source_path + 'buurten voor experts 2e ronde.xlsx', sheet_name='buurten')
# # alleen selecteren kolommen voor scores en oordelen
# sc = sc[['buurtcode','EE','TK']]

# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv(source_path + '2017\\' +  'buurten.csv', encoding='utf-16')

In [57]:
# merge op buurtcode (die van 2019)
# df = pd.merge(sc,buurtendata, on='buurtcode')
df = buurtendata

### Maak splits in test en training set

In [58]:
df.columns

Index(['buurtcode', 'buurtnaam', 'buurtcodeLetters', 'wijkcode', 'wijknaam',
       'gemeentecode', 'gemeentenaam', 'inw', 'bevdh', 'oadh', 'hh', 'inw65',
       'KL14', 'KL16', 'VKL1416', 'RLBRMTR14', 'RLBWON14', 'RLBBEV14',
       'RLBVRZ14', 'RLBVEI14', 'RLBFYS14', 'RLBRMTR16', 'RLBWON16', 'RLBBEV16',
       'RLBVRZ16', 'RLBVEI16', 'RLBFYS16', 'AfstandSupermarkt',
       'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3', 'AantalRest3',
       'AantalKinderopvang1', 'AfstandZwembad', 'AantalMusea10',
       'AantalPodiumk10', 'AantalBios5', 'sleutel', 'wvtotaal', 'wvkoop',
       'wvhuursoc', 'wvhuurov', 'wv<1945', 'wv1945-1970', 'wv1970-1980',
       'wv1980-1990', 'wv1990-2000', 'wv2000-2010', 'wv>=2010', 'aand<1945',
       'aand1945-1970', 'aand1970-1980', 'aand1980-1990', 'aand1990-2000',
       'aand2000-2010', 'aand>=2010', 'aandkoop', 'aandhuursoc', 'aandhuurov',
       'wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2', 'woxGem',
       'woxdelta1jrGem', 'woxdelta5jrGem'

In [59]:
# hernoem RLBBEV naar zonder jaartal, zodat we kunnen vergelijken tussen 2016 en 2018
df.rename(columns={"RLBBEV16":"RLBBEV"}, inplace=True) 

In [61]:
# alle kolommen! behalve enkele omschhrijvingen, onnodige wijkcodes, marktpotentie want String 
# feature_cols = ['woxdelta5jrGem', 'AantalCafe3', 'woxdelta5jr', 'wox',
#        'woxdelta1jrGem', 'AantalBios5', 'woxm2WG']




feature_cols = ['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww',
       'woxdelta5jrGem', 'wox', 'RLBBEV',
       'RLBVRZ16', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10',
       'wv2000-2010', 'AantalBios5']




#zonder wox
#feature_cols = ['buurtcode','inw', 'bevdh', 'oadh', 'hh', 'inw65', 'KL16', 'KL18', 'VKL1618',
#                'RLBRMTR18', 'RLBWON18', 'RLBBEV18', 'RLBVRZ18', 'RLBVEI18', 'RLBFYS18', 
#                'AfstandSupermarkt', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3','AantalRest3', 'AantalKinderopvang1', 
#                'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 
#                'hh19inkLaag', 'hh30inkLaag', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 
#                'aandeelMH19', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen'] 



#X = df[feature_cols]
# y = df[['TK']]

In [62]:
# Normaliseren data
# Creating a scaled feature set
scaler = RobustScaler()
X_2017 = scaler.fit_transform(df[feature_cols])
X_2017 = pd.DataFrame(X_2017)
X_2017.columns = feature_cols

In [63]:
model_2019 = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=250, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# laad model
model_2019 = pickle.load(open("model_backtesting.pkl", "rb"))


In [64]:
y_pred = model_2019.predict(X_2017)

In [65]:
y_pred.shape

(13208,)

In [68]:
X_2017.columns

Index(['AantalCafe3', 'woxdelta5jr', 'woxm2', 'woxww', 'woxdelta5jrGem', 'wox',
       'RLBBEV', 'RLBVRZ16', 'AantalLevensm3', 'woxm2WG', 'aand1945-1970',
       'woxdelta5jrWGabs', 'woxdelta1jrGem', 'AantalPodiumk10', 'wv2000-2010',
       'AantalBios5'],
      dtype='object')

In [67]:
y_pred

array(['c', 'c', 'c', ..., 'c', 'c', 'c'], dtype=object)

In [76]:
# selecteer alleen kolommen om op te vergelijken en key, absolute getallen
X_oud = df[["buurtcode", "woxww"]]
# voeg predicties toe
X_oud['pred_2017'] = list(y_pred)

C:\Users\rondea\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
# laad 2019 data in voor vergelijking
X_nieuw = pd.read_csv(source_path + 'buurten.csv', encoding='utf-16')
# selecteer alleen kolommen om op te vergelijken
X_nieuw = X_nieuw[['buurtcode', 'woxww', 'VKL1618']]

In [105]:
# mergen beiden
X_vgl = pd.merge(X_oud, X_nieuw, on='buurtcode')

# berekenen verschil woxww
X_vgl['delta_woxww'] = X_vgl['woxww_y'] - X_vgl['woxww_x']

In [108]:
X_vgl = X_vgl[X_vgl['delta_woxww'].notnull()]

In [109]:
X_vgl.shape

(6219, 6)

In [103]:
X_vgl.groupby('pred_2017')['delta_woxww'].mean()

pred_2017
a    50674.220963
b    36843.522562
c    47190.228690
Name: delta_woxww, dtype: float64

In [104]:
X_vgl.groupby('pred_2017')['delta_woxww'].count()

pred_2017
a    3883
b    1374
c     962
Name: delta_woxww, dtype: int64

In [96]:

X_vgl = X_vgl[X_vgl['VKL1618'].notnull()]
X_vgl.groupby('pred_2017')['VKL1618'].mean()

pred_2017
a    4.043744
b    4.044248
c    4.045625
Name: VKL1618, dtype: float64